# Long Sequence Classification

- This uses window function in PyTorch.

In [1]:
# Built-in library
import re
import json
from typing import Any, Dict, List, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
import pandas as pd
from rich import print
import torch

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import BertForSequenceClassification, BertTokenizer


model_checkpoint: str = "ProsusAI/finbert"
tokenizer: Any = BertTokenizer.from_pretrained(model_checkpoint)
model: Any = BertForSequenceClassification.from_pretrained(model_checkpoint)

In [3]:
# A very long text!
txt: str = """
I would like to get your all  thoughts on the bond yield increase this week.  I am not worried about the market downturn but the sudden increase in yields. On 2/16 the 10 year bonds yields increased by almost  9 percent and on 2/19 the yield increased by almost 5 percent.

Key Points from the CNBC Article:

* **The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.**
* **Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic.**
* **However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.**

The recent rise in bond yields and U.S. inflation expectations has some investors wary that a repeat of the 2013 “taper tantrum” could be on the horizon.

The benchmark U.S. 10-year Treasury note climbed above 1.3% for the first time since February 2020 earlier this week, while the 30-year bond also hit its highest level for a year. Yields move inversely to bond prices.

Yields tend to rise in lockstep with inflation expectations, which have reached their highest levels in a decade in the U.S., powered by increased prospects of a large fiscal stimulus package, progress on vaccine rollouts and pent-up consumer demand.

The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.

Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic. The Fed and others have maintained supportive tones in recent policy meetings, vowing to keep financial conditions loose as the global economy looks to emerge from the Covid-19 pandemic.

However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.

With central bank support removed, bonds usually fall in price which sends yields higher. This can also spill over into stock markets as higher interest rates means more debt servicing for firms, causing traders to reassess the investing environment.

“The supportive stance from policymakers will likely remain in place until the vaccines have paved a way to some return to normality,” said Shane Balkham, chief investment officer at Beaufort Investment, in a research note this week.

“However, there will be a risk of another ‘taper tantrum’ similar to the one we witnessed in 2013, and this is our main focus for 2021,” Balkham projected, should policymakers begin to unwind this stimulus.

Long-term bond yields in Japan and Europe followed U.S. Treasurys higher toward the end of the week as bondholders shifted their portfolios.

“The fear is that these assets are priced to perfection when the ECB and Fed might eventually taper,” said Sebastien Galy, senior macro strategist at Nordea Asset Management, in a research note entitled “Little taper tantrum.”

“The odds of tapering are helped in the United States by better retail sales after four months of disappointment and the expectation of large issuance from the $1.9 trillion fiscal package.”

Galy suggested the Fed would likely extend the duration on its asset purchases, moderating the upward momentum in inflation.

“Equity markets have reacted negatively to higher yield as it offers an alternative to the dividend yield and a higher discount to long-term cash flows, making them focus more on medium-term growth such as cyclicals” he said. Cyclicals are stocks whose performance tends to align with economic cycles.

Galy expects this process to be more marked in the second half of the year when economic growth picks up, increasing the potential for tapering.

## Tapering in the U.S., but not Europe

Allianz CEO Oliver Bäte told CNBC on Friday that there was a geographical divergence in how the German insurer is thinking about the prospect of interest rate hikes.

“One is Europe, where we continue to have financial repression, where the ECB continues to buy up to the max in order to minimize spreads between the north and the south — the strong balance sheets and the weak ones — and at some point somebody will have to pay the price for that, but in the short term I don’t see any spike in interest rates,” Bäte said, adding that the situation is different stateside.

“Because of the massive programs that have happened, the stimulus that is happening, the dollar being the world’s reserve currency, there is clearly a trend to stoke inflation and it is going to come. Again, I don’t know when and how, but the interest rates have been steepening and they should be steepening further.”

## Rising yields a ‘normal feature’

However, not all analysts are convinced that the rise in bond yields is material for markets. In a note Friday, Barclays Head of European Equity Strategy Emmanuel Cau suggested that rising bond yields were overdue, as they had been lagging the improving macroeconomic outlook for the second half of 2021, and said they were a “normal feature” of economic recovery.

“With the key drivers of inflation pointing up, the prospect of even more fiscal stimulus in the U.S. and pent up demand propelled by high excess savings, it seems right for bond yields to catch-up with other more advanced reflation trades,” Cau said, adding that central banks remain “firmly on hold” given the balance of risks.

He argued that the steepening yield curve is “typical at the early stages of the cycle,” and that so long as vaccine rollouts are successful, growth continues to tick upward and central banks remain cautious, reflationary moves across asset classes look “justified” and equities should be able to withstand higher rates.

“Of course, after the strong move of the last few weeks, equities could mark a pause as many sectors that have rallied with yields look overbought, like commodities and banks,” Cau said.

“But at this stage, we think rising yields are more a confirmation of the equity bull market than a threat, so dips should continue to be bought.”
"""

In [4]:
# Tokenize and prepare for the model a sequence or a pair of sequences.
tokens = tokenizer.encode_plus(txt, add_special_tokens=False, return_tensors="pt")

print(f'Number of tokens: {len(tokens["input_ids"][0])}')
print(f"Tokens: {tokens}")

Token indices sequence length is longer than the specified maximum sequence length for this model (1345 > 512). Running this sequence through the model will result in indexing errors


Number of tokens: 1345

Tokens: {'input_ids': tensor([[1045, 2052, 2066,  ..., 4149, 1012, 1524]]), 'token_type_ids': tensor([[0, 0, 0,  
..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}

### To Do

- Now we have a set of tensors where each tensor contains 1345 tokens. 
- We'll pull out a length of 510 tokens (or less), add the CLS and SEP tokens, then add PAD tokens when needed. 
- To create these tensors of length 510 we need to use the torch.split method.

In [5]:
# Example
A: torch.Tensor = torch.arange(10)
A

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [6]:
# Split the elements in the tensor into chunks each containing at most N elements.
N: int = 4
torch.split(A, N)

(tensor([0, 1, 2, 3]), tensor([4, 5, 6, 7]), tensor([8, 9]))

In [7]:
# Add the special tokens
A: torch.Tensor = torch.cat([torch.Tensor([101]), A, torch.Tensor([102])])
A

tensor([101.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9., 102.])

### To Do

- We still need to add padding to our tensors to push them upto a length of 512, which should only be required for the final chunk. 
- To do this we will build an `if-statement` that checks if the tensor length requires padding, and if so add the correct amount of padding which will be something like required_len = 512 - tensor_len. 
- Again, let's test it on tensor a first:
  - Let's assume we want a max. of 20 tokens per chunk ...

In [8]:
A

tensor([101.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9., 102.])

In [9]:
MAX_TOKEN: int = 20
PADDING_LEN: int = MAX_TOKEN - A.size(0)
PADDING_LEN

8

In [10]:
# If there's a positive padding length
if PADDING_LEN > 0:
    A = torch.cat([A, torch.Tensor([0] * PADDING_LEN)])

A

tensor([101.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9., 102.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.])

### Apply The Logic

In [11]:
# Define target chunk size
CHUNK_SIZE: int = 512
SPECIAL_TOKEN_SIZE: int = 2

# Split into chunks of 510 tokens, we also convert to list (default is tuple which is immutable)
# Substract by two because we wanna add [CLS] and [SEP]. i.e. special tokens.
input_id_chunks: list[int] = list(
    tokens["input_ids"][0].split(CHUNK_SIZE - SPECIAL_TOKEN_SIZE)
)
mask_chunks: list[int] = list(
    tokens["attention_mask"][0].split(CHUNK_SIZE - SPECIAL_TOKEN_SIZE)
)

for chunk in input_id_chunks:
    print(len(chunk))

510

510

325

In [12]:
ATTENTION_PADDING: int = 1
CLS, SEP = (101, 102)

# Loop through each chunk
for idx in range(len(input_id_chunks)):
    # Add CLS and SEP tokens to input IDs
    input_id_chunks[idx] = torch.cat(
        [torch.tensor([CLS]), input_id_chunks[idx], torch.tensor([SEP])]
    )
    # Add attention tokens to attention mask
    mask_chunks[idx] = torch.cat(
        [
            torch.tensor([ATTENTION_PADDING]),
            mask_chunks[idx],
            torch.tensor([ATTENTION_PADDING]),
        ]
    )
    # Get required padding length
    PADDING_LEN: int = CHUNK_SIZE - input_id_chunks[idx].size(0)
    # Check if tensor length satisfies required chunk size
    if PADDING_LEN > 0:
        # If padding length is more than 0, we must add padding
        input_id_chunks[idx] = torch.cat(
            [input_id_chunks[idx], torch.Tensor([0] * PADDING_LEN)]
        )
        mask_chunks[idx] = torch.cat(
            [mask_chunks[idx], torch.Tensor([0] * PADDING_LEN)]
        )

# check length of each tensor
for chunk in input_id_chunks:
    print(len(chunk))
# Print final chunk so we can see 101, 102, and 0 (PAD) tokens are all correctly placed
print(chunk)

512

512

512

tensor([  101.,  2153.,  1010.,  1045.,  2123.,  1521.,  1056.,  2113.,  2043.,
         1998.,  2129.,  1010.,  2021.,  1996.,  3037.,  6165.,  2031.,  2042.,
         9561.,  7406.,  1998.,  2027.,  2323.,  2022.,  9561.,  7406.,  2582.,
         1012.,  1524.,  1001.,  1001.,  4803., 16189.,  1037.,  1520.,  3671.,
         3444.,  1521.,  2174.,  1010.,  2025.,  2035., 18288.,  2024.,  6427.,
         2008.,  1996.,  4125.,  1999.,  5416., 16189.,  2003.,  3430.,  2005.,
         6089.,  1012.,  1999.,  1037.,  3602.,  5958.,  1010., 23724.,  2015.,
         2132.,  1997.,  2647., 10067.,  5656., 14459.,  6187.,  2226.,  4081.,
         2008.,  4803.,  5416., 16189.,  2020.,  2058., 20041.,  1010.,  2004.,
         2027.,  2018.,  2042.,  2474., 12588.,  1996.,  9229., 26632., 23035.,
        17680.,  2005.,  1996.,  2117.,  2431.,  1997., 25682.,  1010.,  1998.,
         2056.,  2027.,  2020.,  1037.,  1523.,  3671.,  3444.,  1524.,  1997.,
         3171.,  7233.,  1012.,  1523.,  2007.,  1996.,  3145.,  6853.,  1997.,
        14200.,  7302.,  2039.,  1010.,  1996.,  9824.,  1997.,  2130.,  2062.,
        10807., 19220.,  1999.,  1996.,  1057.,  1012.,  1055.,  1012.,  1998.,
         7279.,  2102.,  2039.,  5157., 15801.,  2011.,  2152.,  9987., 10995.,
         1010.,  2009.,  3849.,  2157.,  2005.,  5416., 16189.,  2000.,  4608.,
         1011.,  2039.,  2007.,  2060.,  2062.,  3935., 25416., 13490., 14279.,
         1010.,  1524.,  6187.,  2226.,  2056.,  1010.,  5815.,  2008.,  2430.,
         5085.,  3961.,  1523.,  7933.,  2006.,  2907.,  1524.,  2445.,  1996.,
         5703.,  1997., 10831.,  1012.,  2002.,  5275.,  2008.,  1996.,  9561.,
         7406., 10750.,  7774.,  2003.,  1523.,  5171.,  2012.,  1996.,  2220.,
         5711.,  1997.,  1996.,  5402.,  1010.,  1524.,  1998.,  2008.,  2061.,
         2146.,  2004., 17404.,  4897., 12166.,  2024.,  3144.,  1010.,  3930.,
         4247.,  2000., 16356., 10745.,  1998.,  2430.,  5085.,  3961., 17145.,
         1010., 25416., 13490.,  5649.,  5829.,  2408., 11412.,  4280.,  2298.,
         1523., 15123.,  1524.,  1998.,  1041., 15549.,  7368.,  2323.,  2022.,
         2583.,  2000., 19319.,  3020.,  6165.,  1012.,  1523.,  1997.,  2607.,
         1010.,  2044.,  1996.,  2844.,  2693.,  1997.,  1996.,  2197.,  2261.,
         3134.,  1010.,  1041., 15549.,  7368.,  2071.,  2928.,  1037.,  8724.,
         2004.,  2116., 11105.,  2008.,  2031., 24356.,  2007., 16189.,  2298.,
         2058.,  5092., 18533.,  1010.,  2066., 21955.,  1998.,  5085.,  1010.,
         1524.,  6187.,  2226.,  2056.,  1012.,  1523.,  2021.,  2012.,  2023.,
         2754.,  1010.,  2057.,  2228.,  4803., 16189.,  2024.,  2062.,  1037.,
        13964.,  1997.,  1996., 10067.,  7087.,  3006.,  2084.,  1037.,  5081.,
         1010.,  2061., 16510.,  2015.,  2323.,  3613.,  2000.,  2022.,  4149.,
         1012.,  1524.,   102.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,


- It all looks good! Now the final step of placing our tensors back into the dictionary style format we had before.

In [13]:
x1 = torch.tensor([1, 2, 3])
x2 = torch.tensor([4, 5, 6])

# Tuple of tensors!
x = torch.stack((x1, x2))

print(x)

tensor([[1, 2, 3],
        [4, 5, 6]])

In [14]:
input_ids = torch.stack(input_id_chunks)
attention_mask = torch.stack(mask_chunks)

input_dict = {
    "input_ids": input_ids.long(),  # required format
    "attention_mask": attention_mask.int(),  # required format
}
input_dict

{'input_ids': tensor([[  101,  1045,  2052,  ...,  1012,  1523,   102],
         [  101,  1996, 16408,  ...,  2272,  1012,   102],
         [  101,  2153,  1010,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32)}

In [15]:
import torch.nn.functional as F

outputs = model(**input_dict)
probs = F.softmax(outputs[0], dim=-1)

probs

tensor([[0.1384, 0.8145, 0.0471],
        [0.3757, 0.4670, 0.1574],
        [0.7290, 0.2006, 0.0704]], grad_fn=<SoftmaxBackward0>)

In [16]:
import polars as pl

pl.DataFrame(probs.detach().numpy())

column_0,column_1,column_2
f32,f32,f32
0.138411,0.814513,0.047076
0.37567,0.466953,0.157377
0.728986,0.200593,0.070421


In [17]:
# Calculate the average
probs = probs.mean(dim=0)
probs

tensor([0.4144, 0.4940, 0.0916], grad_fn=<MeanBackward1>)